## SQL

###  Задание 1
Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

Задача:  
Дана таблица default.peas:

| Название атрибута | Тип атрибута | Смысловое значение |
|:----------|:----------|:----------|
| st_id    | int   | ID ученика   |
| timest   | timestamp   | Время решения карточки   |
| correct    | bool   | Правильно ли решена горошина?  |
| subject    | text   | Дисциплина, в которой находится горошина   |

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.

NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

###  Задание 2
Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты. 

Задача:  
Дана таблица default.peas(см. выше), default.studs:

| Название атрибута | Тип атрибута | Смысловое значение |
|:----------|:----------|:----------|
| st_id    | int   | ID ученика   |
| test_grp   | text    | Метка ученика в данном эксперименте   |

и default.final_project_check: 

| Название атрибута | Тип атрибута | Смысловое значение |
|:----------|:----------|:----------|
| st_id    | int   | ID ученика   |
| sale_time   | timestamp   | Время покупки  |
| money    | int   | Цена, по которой приобрели данный курс |
| subject    | text   |    |

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:  

* ARPU 
* ARPAU 
* CR в покупку 
* СR активного пользователя в покупку 
* CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике
ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.  

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.  

In [23]:
# импорт библиотек
import pandas as pd
import pandahouse as ph

In [24]:
# объявляем параметры подключения
connection = dict(database='default',
                  host='https://clickhouse.lab.karpov.courses',
                  user='student',
                  password='dpo_python_2020')

### Задание 1
Как будет звучать запрос: за текущий месяц (текущим месяцем будем считать тот, который представлен в базе данных) посчитать количество уникальных студентов (student_id), у которых правильно выполнены задания (correct == 1) и таких заданий не менее 20 строк.

In [17]:
# посмотрим на таблицу
query = '''
    SELECT
        st_id,
        toDateTime(timest) as timest,
        correct,
        subject
    FROM default.peas
    LIMIT 5
'''

peas = ph.read_clickhouse(query, connection=connection)
peas

,st_id,timest,correct,subject
0,100379,2021-10-30 13:32:29,1,Theory of probability
1,100379,2021-10-30 14:11:19,0,Vizualization
2,100379,2021-10-30 15:54:22,1,Theory of probability
3,100379,2021-10-30 16:44:50,1,Vizualization
4,100379,2021-10-30 17:15:05,1,Theory of probability


In [14]:
# запрос
query = """
    SELECT 
        st_id
    FROM default.peas
    WHERE correct == 1 
        AND toMonth(timest)
    GROUP BY st_id
    HAVING count(*) >= 20
"""
df = ph.read_clickhouse(query, connection=connection)
print(f'Количество усердных студентов = {df.shape[0]} человек.')

Количество усердных студентов = 136 человек.


### Задание 2
Посмотрим на другие таблицы

In [18]:
query = '''
    SELECT
        st_id,
        test_grp
    FROM default.studs
    LIMIT 5
'''

studs = ph.read_clickhouse(query, connection=connection)
studs

,st_id,test_grp
0,100379,pilot
1,101432,control
2,104818,pilot
3,104885,pilot
4,104966,pilot


In [20]:
query = '''
    SELECT
        st_id,
        toDateTime(sale_time) as sale_time,
        money,
        subject
    FROM default.final_project_check
    LIMIT 5
'''

final_project_check = ph.read_clickhouse(query, connection=connection)
final_project_check

,st_id,sale_time,money,subject
0,101432,2021-10-31 04:44:32,85000,Math
1,101432,2021-10-31 12:43:50,65000,Vizualization
2,104885,2021-10-30 17:05:55,65000,Vizualization
3,104885,2021-10-30 22:49:33,75000,Statistics
4,106464,2021-10-31 13:17:13,85000,Math


Запрос:  
ARPU = выручка / количество пользователей    
ARPAU = выручка от активных пользователей / количество активных пользователей  
CR = количество клиентов / количество пользователей


In [22]:
# составим запрос 
query = '''
    SELECT 
        b.test_grp AS group,
        SUM(c.money) / COUNT(DISTINCT(b.st_id)) AS ARPU,
        SUM(CASE WHEN a.sum_correct > 10 THEN c.money END) / COUNT(DISTINCT(CASE WHEN a.sum_correct > 10 THEN b.st_id END)) AS ARPAU,
        COUNT(DISTINCT CASE WHEN c.money != 0 THEN c.st_id END) / COUNT(DISTINCT b.st_id) * 100 AS CR,
        COUNT(DISTINCT CASE WHEN a.sum_correct > 10 and c.money != 0  THEN b.st_id END) / COUNT(DISTINCT CASE WHEN a.sum_correct > 10 THEN b.st_id END) * 100 AS CR_active_users,
        COUNT(DISTINCT CASE WHEN c.subject = 'Math' and a.sum_math_correct >= 2 and c.money != 0  THEN b.st_id END) / COUNT(DISTINCT CASE WHEN a.sum_math_correct >= 2 THEN b.st_id END) * 100 AS CR_math
    FROM
        (SELECT
            DISTINCT(st_id) AS st_id,
            sum(correct) AS sum_correct,
            sum(CASE WHEN subject == 'Math' THEN correct END) AS sum_math_correct
        FROM
            default.peas
        GROUP BY st_id
        ) a
    FULL JOIN default.studs as b ON a.st_id=b.st_id
    FULL JOIN default.final_project_check as c ON b.st_id=c.st_id
    GROUP BY group
    '''

metrics = ph.read_clickhouse(query, connection=connection)
metrics

,group,ARPU,ARPAU,CR,CR_active_users,CR_math
0,control,4540.983607,10393.700787,4.918033,11.023622,6.122449
1,pilot,11508.474576,29739.583333,10.847458,26.041667,9.523810


Заключение: в тестовой группе наблюдается значительный рост всех метрик